In [1]:
import os
import time
import random
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ===============================================================
# [설정] 부산항대교 정밀 분석용 키워드 (쌍따옴표 포함)
# ===============================================================
TARGET_PLACE_NAME = "영화의전당"
TARGET_COUNT = 500

# 영화의전당의 특징인 '빅루프', '야경', '부산국제영화제(BIFF)', '센텀시티' 등을 조합했습니다.
RELAY_KEYWORDS = [
    '"영화의전당" "여행"',        # 기본 여행 정보
    '"영화의전당" "야경"',        # 지붕(빅루프) 조명 관련
    '"영화의전당" "BIFF"',        # 부산국제영화제 관련 현장감
    '"영화의전당" "빅루프"',      # 세계 최대 지붕, 건축미 관련
    '"영화의전당" "전시"',        # 내부 문화 행사 정보
    '"센텀시티" "영화의전당"',    # 지역구 기반 위치 정보
    '"나루공원" "영화의전당"'     # 근처 연계 관광지 정보
]

SAVE_FILENAME = f"naver_blog_{TARGET_PLACE_NAME}_manual_exact.csv"
# ===============================================================

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

def main():
    driver = setup_driver()
    wait = WebDriverWait(driver, 10)

    # 기존 데이터 로드
    existing_links = set()
    if os.path.isfile(SAVE_FILENAME):
        with open(SAVE_FILENAME, 'r', encoding='utf-8-sig') as rf:
            reader = csv.reader(rf)
            try:
                next(reader, None)
                for row in reader:
                    if len(row) > 5: existing_links.add(row[5])
            except: pass
    
    f = open(SAVE_FILENAME, 'a', encoding='utf-8-sig', newline='')
    writer = csv.writer(f)
    if len(existing_links) == 0:
        writer.writerow(['주제', '검색어', '제목', '본문', '날짜', '링크'])

    print(f"\n📂 [정밀 수집] 현재 확보: {len(existing_links)}개 / 목표: {TARGET_COUNT}개")

    try:
        driver.get("https://section.blog.naver.com/Search/Post.naver")
        time.sleep(2)

        for keyword in RELAY_KEYWORDS:
            if len(existing_links) >= TARGET_COUNT:
                print(f"\n🎉 목표 달성! 퇴근하세요!")
                break

            print(f"\n" + "="*60)
            print(f"👉 [요청] 브라우저에 아래 검색어를 **정확히** 입력해주세요.")
            print(f"   검색어:  {keyword}")
            print(f"   (주의: 양옆의 쌍따옴표 \" \" 도 반드시 입력해야 합니다!)")
            print("="*60)
            print("1. 브라우저에서 검색하고 결과가 나오면,")
            print("2. **터미널을 클릭**하고 [Enter]를 누르세요.")
            
            # 사용자 입력 대기 (여기서 차단 방지 효과 발생)
            input(f"   ( '{keyword}' 검색 완료 후 엔터...)")

            print(f"🚀 '{keyword}' 수집 시작...")
            
            collected_links = []
            page_num = 1
            consecutive_dupes = 0
            
            # --- 링크 수집 ---
            while True:
                try:
                    links = driver.find_elements(By.CSS_SELECTOR, "a.desc_inner")
                    new_cnt = 0
                    for link in links:
                        url = link.get_attribute("href")
                        if url and url not in existing_links and url not in collected_links:
                            collected_links.append(url)
                            new_cnt += 1
                        elif url in existing_links:
                            consecutive_dupes += 1
                    
                    print(f"   └ {page_num}페이지: 신규 {new_cnt}개 (누적 중복: {consecutive_dupes})")

                    # 탈출 조건
                    if new_cnt == 0: 
                        print("   ✋ 더 이상 새로운 글이 없습니다. 다음 검색어로!")
                        break
                    
                    if len(collected_links) >= 150:
                        print("   ✨ 150개 확보 완료. 다음 검색어로!")
                        break

                    # 다음 페이지
                    try:
                        next_btn = driver.find_element(By.CSS_SELECTOR, "a.button_next")
                        driver.execute_script("arguments[0].click();", next_btn)
                        page_num += 1
                        time.sleep(random.uniform(0.8, 1.2))
                    except:
                        print("   ⚠️ 마지막 페이지입니다.")
                        break
                except: break

            # --- 본문 수집 ---
            if collected_links:
                print(f"🚀 본문 {len(collected_links)}개 긁어오는 중...")
                for i, post_url in enumerate(collected_links):
                    try:
                        driver.get(post_url)
                        time.sleep(random.uniform(1.5, 2.0))

                        if (i + 1) % 50 == 0:
                            print("     ☕ 휴식...")
                            time.sleep(5)

                        try: driver.switch_to.frame("mainFrame")
                        except: pass

                        title = ""
                        try: title = driver.find_element(By.CSS_SELECTOR, ".se-title-text, .h_title").text
                        except: pass

                        content = ""
                        try: content = driver.find_element(By.CSS_SELECTOR, ".se-main-container, #postViewArea").text
                        except: pass
                        
                        date = ""
                        try: date = driver.find_element(By.CSS_SELECTOR, ".se_publishDate, .post_date").text
                        except: pass

                        if title and content:
                            content = content.replace("\n", " ").replace("\t", " ")
                            title = title.replace("\n", " ")
                            
                            writer.writerow([TARGET_PLACE_NAME, keyword, title, content, date, post_url])
                            f.flush()
                            existing_links.add(post_url)
                            print(f"     [{len(existing_links)}/{TARGET_COUNT}] 저장됨")
                        
                        if len(existing_links) >= TARGET_COUNT:
                            break
                    except: continue
            
            # 검색창 초기화
            driver.get("https://section.blog.naver.com/Search/Post.naver")
            time.sleep(1)

    except Exception as e:
        print(f"[오류] {e}")

    finally:
        f.close()
        driver.quit()
        print(f"\n[종료] 총 {len(existing_links)}개 저장 완료.")

if __name__ == "__main__":
    main()


📂 [정밀 수집] 현재 확보: 0개 / 목표: 500개

👉 [요청] 브라우저에 아래 검색어를 **정확히** 입력해주세요.
   검색어:  "영화의전당" "여행"
   (주의: 양옆의 쌍따옴표 " " 도 반드시 입력해야 합니다!)
1. 브라우저에서 검색하고 결과가 나오면,
2. **터미널을 클릭**하고 [Enter]를 누르세요.
🚀 '"영화의전당" "여행"' 수집 시작...
   └ 1페이지: 신규 7개 (누적 중복: 0)
   └ 2페이지: 신규 7개 (누적 중복: 0)
   └ 3페이지: 신규 7개 (누적 중복: 0)
   └ 4페이지: 신규 7개 (누적 중복: 0)
   └ 5페이지: 신규 7개 (누적 중복: 0)
   └ 6페이지: 신규 7개 (누적 중복: 0)
   └ 7페이지: 신규 7개 (누적 중복: 0)
   └ 8페이지: 신규 7개 (누적 중복: 0)
   └ 9페이지: 신규 7개 (누적 중복: 0)
   └ 10페이지: 신규 7개 (누적 중복: 0)
   └ 11페이지: 신규 7개 (누적 중복: 0)
   └ 12페이지: 신규 7개 (누적 중복: 0)
   └ 13페이지: 신규 7개 (누적 중복: 0)
   └ 14페이지: 신규 7개 (누적 중복: 0)
   └ 15페이지: 신규 7개 (누적 중복: 0)
   └ 16페이지: 신규 0개 (누적 중복: 0)
   ✋ 더 이상 새로운 글이 없습니다. 다음 검색어로!
🚀 본문 105개 긁어오는 중...
     [1/500] 저장됨
     [2/500] 저장됨
     [3/500] 저장됨
     [4/500] 저장됨
     [5/500] 저장됨
     [6/500] 저장됨
     [7/500] 저장됨
     [8/500] 저장됨
     [9/500] 저장됨
     [10/500] 저장됨
     [11/500] 저장됨
     [12/500] 저장됨
     [13/500] 저장됨
     [14/500] 저장됨
     [15/500] 저장됨
     [16/500] 저장됨
  

In [2]:
import os
import requests
import csv
import time

# ===============================================================
# [설정] 발급받은 API 키 입력
# ===============================================================
NAVER_CLIENT_ID = "Yq4GzoTBSEeaz00aW0XP"
NAVER_CLIENT_SECRET = "HrI_dsHAB8"

TARGET_PLACE_NAME = "벡스코"
TARGET_COUNT = 500

# 벡스코의 특성에 맞춘 검색 키워드 조합
RELAY_KEYWORDS = [
    '벡스코 박람회 후기', '벡스코 전시회 일정', '벡스코 주차 정보', 
    '벡스코 컨벤션홀', '벡스코 오디토리움 콘서트', '해운대 벡스코 맛집', 
    '벡스코 지스타(G-STAR)', '벡스코 베이비페어'
]

SAVE_FILENAME = f"naver_blog_{TARGET_PLACE_NAME}_api.csv"
# ===============================================================

def get_blog_search(keyword, display=100, start=1):
    url = "https://openapi.naver.com/v1/search/blog.json"
    headers = {
        "X-Naver-Client-Id": NAVER_CLIENT_ID,
        "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
    }
    params = {
        "query": keyword,
        "display": display,
        "start": start,
        "sort": "sim" # 유사도순
    }
    
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Error Code: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ 요청 중 오류 발생: {e}")
        return None

def main():
    existing_links = set()
    # 기존 파일이 있다면 중복 방지를 위해 링크 로드
    if os.path.isfile(SAVE_FILENAME):
        with open(SAVE_FILENAME, 'r', encoding='utf-8-sig') as rf:
            reader = csv.reader(rf)
            next(reader, None)
            for row in reader:
                if len(row) > 5: existing_links.add(row[5])

    # 파일 모드 결정 (쓰기 또는 추가)
    file_mode = 'a' if os.path.isfile(SAVE_FILENAME) else 'w'
    with open(SAVE_FILENAME, file_mode, encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        if file_mode == 'w':
            writer.writerow(['주제', '검색어', '제목', '본문(요약)', '날짜', '링크'])

        print(f"📂 [벡스코 데이터 수집] 현재: {len(existing_links)}개 / 목표: {TARGET_COUNT}개")

        for keyword in RELAY_KEYWORDS:
            if len(existing_links) >= TARGET_COUNT: break
            
            print(f"\n🔍 검색어: {keyword}")
            
            # 한 키워드당 최대 1000개까지 조회 가능 (100개씩 10번)
            for start_index in range(1, 1001, 100):
                if len(existing_links) >= TARGET_COUNT: break
                
                data = get_blog_search(keyword, display=100, start=start_index)
                if not data or 'items' not in data or not data['items']:
                    break
                
                new_cnt = 0
                for item in data['items']:
                    link = item['link']
                    if link not in existing_links:
                        # 제목과 요약에서 HTML 태그 제거
                        title = item['title'].replace('<b>', '').replace('</b>', '').replace('&quot;', '"')
                        desc = item['description'].replace('<b>', '').replace('</b>', '').replace('&quot;', '"')
                        
                        writer.writerow([TARGET_PLACE_NAME, keyword, title, desc, item['postdate'], link])
                        existing_links.add(link)
                        new_cnt += 1
                        
                        if len(existing_links) >= TARGET_COUNT: break
                
                print(f"   - {start_index}번~ 데이터 처리 완료 (신규 {new_cnt}개)")
                time.sleep(0.1) # 서버 부하 방지
                
                if new_cnt == 0: break # 더 이상 새로운 데이터가 없으면 다음 키워드로

    print(f"\n✨ 수집 종료: 총 {len(existing_links)}개 저장됨 ({SAVE_FILENAME})")

if __name__ == "__main__":
    main()

📂 [벡스코 데이터 수집] 현재: 0개 / 목표: 500개

🔍 검색어: 벡스코 박람회 후기
   - 1번~ 데이터 처리 완료 (신규 100개)
   - 101번~ 데이터 처리 완료 (신규 100개)
   - 201번~ 데이터 처리 완료 (신규 100개)
   - 301번~ 데이터 처리 완료 (신규 100개)
   - 401번~ 데이터 처리 완료 (신규 100개)

✨ 수집 종료: 총 500개 저장됨 (naver_blog_벡스코_api.csv)
